# IO: Reading and Writing Files

MolPy supports multiple file formats for structures and force fields. Complete IO guide.

## Table of Contents

1. [Overview](#overview)
2. [Structure Formats](#structures)
3. [Force Field Formats](#forcefields)
4. [LAMMPS Integration](#lammps)

## Overview

### Design Principle

**Explicit functions, not magic**

- `read_xyz()` reads XYZ
- `write_lammps_data()` writes LAMMPS DATA
- No guessing from extensions

In [1]:
from molpy.io import read_xyz, write_lammps_data

# Read
# frame = read_xyz('mol.xyz')

# Write
# write_lammps_data(frame, 'system.data')

print('IO functions available')

IO functions available


## Structure Formats

- **XYZ**: Simple coordinates
- **PDB**: Protein Data Bank
- **LAMMPS DATA**: Full topology

All return Frame objects.

## See Also

- [Data Formats](data_formats.ipynb)
- [Force Field Formats](forcefield_formats.ipynb)
- [LAMMPS Integration](lammps_integration.ipynb)